In [1]:
n_test = 100
K = 1000

import torch

from mcce import MCCE

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
from carla.data.catalog import OnlineCatalog
import pandas as pd

train_path = "/nr/samba/user/anr/pkg/MCCE/Datasets/Adult/adult.data"
test_path = "/nr/samba/user/anr/pkg/MCCE/Datasets/Adult/adult.test"
train = pd.read_csv(train_path, sep=", ", header=None, names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])
test = pd.read_csv(test_path, skiprows=1, sep=", ", header=None, names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])

df = train

# df = pd.concat([train, test], axis=0, ignore_index=True)
# df.to_csv("/nr/samba/user/anr/pkg/MCCE/Datasets/Adult/Adult_train_test.csv")

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


In [16]:
mapping = {'>50K': '>50K', '>50K.': '>50K', '<=50K': '<=50K', '<=50K.': '<=50K'}

df['income'] = [mapping[item] for item in df['income']]

In [17]:
feature = 'workclass'
d = df.groupby([feature]).size().sort_values(ascending=False)
for i, ind in enumerate(d):
    d[i] = i
mapping = d.to_dict()
df[feature] = [mapping[item] for item in df[feature]]

feature = 'education'
d = df.groupby([feature]).size().sort_values(ascending=False)
for i, ind in enumerate(d):
    d[i] = i
mapping = d.to_dict()
df[feature] = [mapping[item] for item in df[feature]]

feature = 'marital-status'
d = df.groupby([feature]).size().sort_values(ascending=False)
for i, ind in enumerate(d):
    d[i] = i
mapping = d.to_dict()
df[feature] = [mapping[item] for item in df[feature]]

feature = 'occupation'
d = df.groupby([feature]).size().sort_values(ascending=False)
for i, ind in enumerate(d):
    d[i] = i
mapping = d.to_dict()
df[feature] = [mapping[item] for item in df[feature]]

feature = 'relationship'
d = df.groupby([feature]).size().sort_values(ascending=False)
for i, ind in enumerate(d):
    d[i] = i
mapping = d.to_dict()
df[feature] = [mapping[item] for item in df[feature]]

feature = 'sex'
d = df.groupby([feature]).size().sort_values(ascending=False)
for i, ind in enumerate(d):
    d[i] = i
mapping = d.to_dict()
df[feature] = [mapping[item] for item in df[feature]]

feature = 'race'
d = df.groupby([feature]).size().sort_values(ascending=False)
for i, ind in enumerate(d):
    d[i] = i
mapping = d.to_dict()
df[feature] = [mapping[item] for item in df[feature]]

feature = 'native-country'
d = df.groupby([feature]).size().sort_values(ascending=False)
for i, ind in enumerate(d):
    d[i] = i
mapping = d.to_dict()
df[feature] = [mapping[item] for item in df[feature]]

feature = 'income'
d = df.groupby([feature]).size().sort_values(ascending=False)
for i, ind in enumerate(d):
    d[i] = i
mapping = d.to_dict()
df[feature] = [mapping[item] for item in df[feature]]

df.drop(['education'], inplace=True, axis=1)

In [18]:
df.to_csv("/nr/samba/user/anr/pkg/MCCE/Datasets/Adult/train_not_normalized_data_from_carla.csv", index=False)

In [ ]:
# df[['workclass', 'occupation', 'relationship', 'native-country']].groupby(by = 'workclass').size()

In [ ]:
continuous = ['age',
    'fnlwgt',
    'education-num',
    'capital-gain',
    'hours-per-week',
    'capital-loss']

categorical = ['marital-status',
'native-country',
'occupation',
'race',
'relationship',
'sex',
'workclass', 
'income']

from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler().fit(df[continuous])
df_scaled = scaler.transform(df[continuous])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = continuous

In [ ]:
df_scaled = pd.concat([df_scaled, df[categorical]], axis=1) # axis = 0 is index


In [ ]:
df_scaled.to_csv("/nr/samba/user/anr/pkg/MCCE/Datasets/Adult/for_CsvCatalog.csv", index=False)

In [ ]:
from carla.data.catalog import CsvCatalog

continuous = ["age", "fnlwgt", "education-num", "capital-gain", "hours-per-week", "capital-loss"]
categorical = ["marital-status", "native-country", "occupation", "race", "relationship", "sex", "workclass"]
immutable = ["age", "sex"]

dataset = CsvCatalog(file_path="/nr/samba/user/anr/pkg/MCCE/Datasets/Adult/for_CsvCatalog.csv",
                     continuous=continuous,
                     categorical=categorical,
                     immutables=immutable,
                     target='income',
                     scaling_method=None,
                     encoding_method=None,
                     )

# dataset.df

In [ ]:
from carla.models.catalog import MLModelCatalog
import torch
torch.manual_seed(0)

ml_model = MLModelCatalog(
        dataset, 
        model_type="ann", 
        load_online=False, 
        backend="pytorch"
    )


ml_model.train(
learning_rate=0.002,
epochs=20,
batch_size=1024,
hidden_size=[18, 9, 3],
force_train=True, # don't forget to add this or it might load an older model from disk
)

In [ ]:
from carla.models.negative_instances import predict_negative_instances

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:n_test]
# test_factual_inverse = dataset.inverse_transform(test_factual)

y_col = dataset.target
features_and_response = dataset.df.columns
cont_feat = dataset.continuous
cat_feat = [x for x in features_and_response if x not in cont_feat] #  these have new names since encode_normalize_order_factuals()

fixed_features = ['age', 'sex']

#  Create dtypes for MCCE()
dtypes = dict([(x, "float") for x in cont_feat])
for x in cat_feat:
    dtypes[x] = "category"
df = (dataset.df).astype(dtypes)

import time
start = time.time()
# (3) Fit MCCE object
print("Fitting MCCE model...")
mcce = MCCE(fixed_features=fixed_features, model=ml_model, seed=1)
mcce.fit(df.drop(y_col, axis=1), dtypes)
print("Generating counterfactuals with MCCE...")
synth_df = mcce.generate(test_factual.drop(y_col, axis=1), k=K)

In [ ]:
# (4) Postprocess generated counterfactuals
print("Postprocessing counterfactuals with MCCE...")
mcce.postprocess(df, synth_df, test_factual, y_col, scaler=dataset.inverse_transform, cutoff=0.5)
timing = time.time() - start
print(timing)

mcce.results_sparse['time (seconds)'] = timing

# 20 minutes for give_me_some_credit

In [ ]:
mcce.results_sparse.to_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/adult_mcce_results_raw_data_k_{K}.csv")

In [ ]:
# print(mcce.results_sparse.)
print(mcce.results_sparse.L0.mean())
print(mcce.results_sparse.L1.mean())
print(mcce.results_sparse.L2.mean())

In [ ]:
results_sparse = mcce.results_sparse
results_sparse.index.rename('index', inplace=True)
results_sparse.groupby('index').size().sort_values(ascending=False)
#results_sparse.loc[64].iloc[0]#[[dataset.categorical + dataset.continuous]]

In [ ]:
# df_decoded_cfs = pd.DataFrame(scaler.inverse_transform(df_cfs.copy()[continuous]))
# df_decoded_cfs.columns = continuous
# df_decoded_cfs.index = df_cfs.index
# df_decoded_cfs = pd.concat([df_decoded_cfs, df_cfs.copy()[categorical]], axis=1)

# df_factuals = pd.DataFrame(scaler.inverse_transform(test_repeated.copy()[continuous]))
# df_factuals.columns = continuous
# df_factuals.index = test_repeated.index
# df_factuals = pd.concat([df_factuals, test_repeated.copy()[categorical]], axis=1)


In [ ]:
print(mcce.results_sparse.feasibility.mean())